In [3]:
import pandas as pd
import numpy as np
import requests
import re
import os
from time import sleep
from tqdm import tqdm
from functools import reduce

import datetime

In [22]:
BASE_GOOGLE_FINANCE_URL = "https://api.polygon.io/v2/aggs/ticker/{TICKER}/range/{INTERVAL}/" +\
                          "{RANGE}/{FROM_DATE}/{TO_DATE}?limit=50000&apiKey=MNxlbyNXPpKmAki1jcScqmXxzPvdXWYT"

# TOKENS = [
#     "AAPL", "GOOG", "MSFT", "TSLA", "NVDA", "AMZN",
#     "FB", "BABA", "CRM", "INTC", "AMD", "PYPL", "ATVI",
#     "EA", "TTD", "MTCH", "ZG", "YELP"
# ]
TOKENS = ["AAPL"]

TIME_INTERVAL = 15
TIME_UNITS = "minute"
FROM_DATE = pd.Timestamp("2018-02-18").date()
TO_DATE = pd.Timestamp("2022-04-29").date()

DATASET_NAME = "AAPL"

FULL_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}.csv"
TRAIN_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}_train.csv"
TEST_RESULT_PATH = f"prices/{DATASET_NAME}_{TIME_INTERVAL}{TIME_UNITS}_test.csv"

In [23]:
def get_url(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    """
    Build url for Google Finance API
    """
    url = re.sub("{TICKER}", token, BASE_GOOGLE_FINANCE_URL)
    url = re.sub("{INTERVAL}", str(interval_len), url)
    url = re.sub("{RANGE}", interval, url)
    url = re.sub("{FROM_DATE}", str(from_date), url)
    url = re.sub("{TO_DATE}", str(to_date), url)
    return url

In [24]:
def get_data(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    TIMEOUT = 20
    current_date = from_date
    previous_date = current_date
    dfs = []
    while current_date <= to_date:
        request_url = get_url(token, interval_len, interval, current_date, to_date)
        data = requests.get(url=request_url)
        if data.json()["status"] == "ERROR":
            print("Waiting for", TIMEOUT, "seconds")
            for _ in tqdm(range(TIMEOUT)):
                sleep(1)
        else:
            try:
                current_df = pd.DataFrame(data.json()["results"])
            except:
                pass
            current_date = (pd.Timestamp(current_df["t"].max(), unit="ms") + pd.DateOffset(days=1)).date()
            print("Current date:", current_date)
            print("Get DataFrame of shape", current_df.shape)
            dfs.append(current_df)
    result_df = pd.concat(dfs)
    result_df.drop_duplicates(inplace=True)
    return result_df

In [25]:
dfs = []
for token in TOKENS:
    data = get_data(
        token=token,
        interval_len=TIME_INTERVAL,
        interval=TIME_UNITS,
        from_date=FROM_DATE, 
        to_date=TO_DATE
    )

    data["t"] = data["t"].apply(lambda row: pd.Timestamp(row, unit="ms"))
    data["ticker"] = token
    dfs.append(data)

Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Current date: 2020-08-11
Get DataFrame of shape (4411, 8)
Current date: 2020-10-30
Get DataFrame of shape (3617, 8)
Current date: 2021-01-27
Get DataFrame of shape (3696, 8)
Current date: 2021-04-21
Get DataFrame of shape (3684, 8)
Current date: 2021-07-21
Get DataFrame of shape (3972, 8)
Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Current date: 2021-10-20
Get DataFrame of shape (4048, 8)
Current date: 2022-01-19
Get DataFrame of shape (3902, 8)
Current date: 2022-04-15
Get DataFrame of shape (3870, 8)
Current date: 2022-04-30
Get DataFrame of shape (639, 8)


In [26]:
RENAME_COLUMNS = {
    "v": "volume",
    "o": "open",
    "c": "close",
    "h": "high",
    "l": "low",
    "t": "date"
}
KEEP_COLUMNS = ["volume", "open", "close", "high", "low", "date", "ticker"]

result_df = pd.concat(dfs).rename(RENAME_COLUMNS, axis=1)[KEEP_COLUMNS].reset_index(drop=True)
print(result_df.shape)
result_df

(31839, 7)


,volume,open,close,high,low,date,ticker
0,13828.0,72.0625,72.000,72.0625,71.7875,2020-05-04 07:51:00,AAPL
1,6856.0,72.0000,71.975,72.0000,71.9050,2020-05-04 08:06:00,AAPL
2,7012.0,72.0100,71.920,72.0150,71.9200,2020-05-04 08:21:00,AAPL
3,1064.0,71.9725,72.000,72.0000,71.9725,2020-05-04 08:36:00,AAPL
4,20052.0,72.0625,71.875,72.0625,71.8750,2020-05-04 08:51:00,AAPL
...,...,...,...,...,...,...,...
31834,13719.0,157.8500,157.900,157.9300,157.8500,2022-04-29 22:45:00,AAPL
31835,10466.0,157.8701,157.900,157.9300,157.8600,2022-04-29 23:00:00,AAPL
31836,17632.0,157.9000,157.990,157.9900,157.9000,2022-04-29 23:15:00,AAPL
31837,20486.0,157.9600,158.050,158.1500,157.9600,2022-04-29 23:30:00,AAPL


In [27]:
TEST_RATIO = 0.2
test_size = int(result_df.shape[0] * TEST_RATIO)
data_train = result_df.iloc[:-test_size]
data_test = result_df.iloc[-test_size:]

result_df.to_csv(FULL_RESULT_PATH)
data_train.to_csv(TRAIN_RESULT_PATH)
data_test.to_csv(TEST_RESULT_PATH)

print(data_train.shape, data_test.shape)

(25472, 7) (6367, 7)


In [28]:
train_dates = data_train.date.apply(lambda date: pd.Timestamp(date))
test_dates = data_test.date.apply(lambda date: pd.Timestamp(date))

print(f"Train is from {train_dates.min()} to {train_dates.max()}")
print(f"Test is from {test_dates.min()} to {test_dates.max()}")

Train is from 2020-05-04 07:51:00 to 2021-12-06 13:45:00
Test is from 2021-12-06 14:00:00 to 2022-04-29 23:45:00
